# Load dataset and pre-processing

In [5]:
!pip install kagglehub 
!pip install scikit-learn
!pip install nltk
!pip install WordCloud
from wordcloud import WordCloud


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
df_summary = 0

In [8]:
# This code is downloading the notebook from GitHub and running it
import requests
from pathlib import Path
url = "https://raw.githubusercontent.com/nbakas/NLP/refs/heads/main/02-Preprocessing.ipynb"
filename = url.split("/")[-1]
local_path = Path.cwd() / filename
if not local_path.exists():
    response = requests.get(url)
    response.raise_for_status()
    local_path.write_bytes(response.content)
%run {str(local_path)}

Exception: File `'/workspaces/dereeNLP/midterm'` not found.

In [ ]:
df_summary

0               good quality dog food
1                       not advertise
2                         delight say
3                      cough medicine
4                         great taffy
                     ...             
568449                    not without
568450                   disappointed
568451               perfect maltipoo
568452    favorite train reward treat
568453                    great honey
Name: Summary, Length: 568454, dtype: object

# Libraries

In [ ]:
# We will use gensim library for topic modeling
# Import corpora module for document processing
from gensim import corpora
# Import LdaMulticore for parallel LDA implementation
from gensim.models.ldamulticore import LdaMulticore
# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Convert df_summary to list of texts

In [ ]:
my_texts = df_summary.astype(str).tolist()
my_texts[:5]

['good quality dog food',
 'not advertise',
 'delight say',
 'cough medicine',
 'great taffy']

# Tokenize the texts

In [ ]:
processed_texts = [my_text.split() for my_text in my_texts]
processed_texts[:5]

[['good', 'quality', 'dog', 'food'],
 ['not', 'advertise'],
 ['delight', 'say'],
 ['cough', 'medicine'],
 ['great', 'taffy']]

# Create a dictionary mapping words to their IDs

In [ ]:
# This cell is creating a dictionary of words and their IDs from the processed texts.
my_dictionary = corpora.Dictionary(processed_texts)
my_dictionary

In [ ]:
# Print 10 random items from the dictionary to understand its structure
print("10 random items from the dictionary:")
import random
random_ids = random.sample(list(my_dictionary.keys()), 10)
for word_id in random_ids:
    print(f"Word ID {word_id}: {my_dictionary[word_id]}")

10 random items from the dictionary:
Word ID 981: horrid
Word ID 44994: sleepies
Word ID 2632: term
Word ID 34082: zotz!!!
Word ID 29204: veja
Word ID 54632: soya
Word ID 55579: from...louisiana?!
Word ID 51525: sobroso!
Word ID 28974: tubs!
Word ID 11312: product...no


In [ ]:
len(my_dictionary)

57360

# Filter out extreme values

In [ ]:
# Filter out extreme values (optional) to improve LDA performance and quality
# no_below=100: Remove words that appear in fewer than 100 documents (rare terms)
#   - Removes noise and very specific terms that don't help identify general topics
# no_above=0.1: Remove words that appear in more than 10% of documents (too common)
#   - Removes overly common words that appear across many topics and don't help differentiate
# This filtering reduces my_dictionary size, speeds up computation, and helps LDA focus on meaningful topic-specific words
my_dictionary.filter_extremes(no_below=10, no_above=0.1)
my_dictionary

In [1]:
len(my_dictionary)

NameError: name 'my_dictionary' is not defined

# Create a document-term matrix

In [12]:
# This cell is creating a "bag-of-words" representation of your processed texts using the Gensim library.
# In the following line, `my_corpus = [my_dictionary.doc2bow(text) for text in processed_texts]`, each document in `processed_texts` is converted to a bag-of-words format using `doc2bow()`.
my_corpus = [my_dictionary.doc2bow(text) for text in processed_texts]

In [ ]:
my_corpus[:10]
# Each tuple (word_id, frequency) represents a word by its dictionary ID and how many times it appears in that document.

In [ ]:
processed_texts[:10]

In [ ]:
# print the first 10 documents in the corpus, their words and their IDs
for doc_id, doc in enumerate(my_corpus[:10]):
    print(f"Document {doc_id+1}:")
    for word_id, freq in doc:
        word = my_dictionary[word_id]
        print(f"  Word ID {word_id} ('{word}'): Frequency {freq}")

---

**Note on Test Set Evaluation**

Since **LDA is an unsupervised learning method**, we often use the full dataset to build the model — there are no labels to predict, as in supervised learning.

However, using a **test set** can still be very useful.
It helps:

* Evaluate how well the model **generalizes to new, unseen data**
* Compare models with different **numbers of topics**
* Avoid **overfitting** to the training data

---

# Set LDA parameters

In [16]:
num_topics = 10  # Number of topics to be extracted
my_passes = 10 # Number of my_passes of the corpus through the model during training. More my_passes means better accuracy but longer runtime
workers = 4  # Number of worker processes for parallel computing

# Train the LDA model

In [17]:
# It will take ~10 minutes to train the model if the dictionary is not filtered.
# https://radimrehurek.com/gensim/models/ldamulticore.html
lda_model = LdaMulticore(
    corpus=my_corpus, # The document-term list we created earlier
    id2word=my_dictionary, # Maps word IDs to actual words for interpretable output
    num_topics=num_topics, # Number of topics to extract 
    passes=my_passes, # Number of training my_passes through the corpus 
    workers=workers, # Number of parallel processes to use 
    alpha='symmetric', # Topic distribution prior - 'symmetric' gives equal probability to all topics initially
    eta='auto' # Word distribution prior (influences how words are distributed across topics). 'auto' lets the model learn optimal word weights. β in notes.
)

# Evaluate LDA model performance

## Coherence score

- A coherence score tells us if the words in a topic appear in similar texts or contexts.
- For example, in a good topic like: ["dog", "cat", "pet", "animal"], these words often show up in the same documents.
- It takes values between 0 and 1, with 1 being the highest coherence. Typical values are between 0.3 and 0.6.

In [ ]:
# https://radimrehurek.com/gensim/models/coherencemodel.html
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, # LDA model
                                     texts=processed_texts, # list of texts, each text is a list of words
                                     dictionary=my_dictionary, # dictionary of words and their IDs
                                     coherence='c_v', # coherence measure, c_v id defined as the average pointwise mutual information of all word pairs in a topic
                                     topn=20 # number of words to consider for coherence score
                                     )
coherence_score = coherence_model_lda.get_coherence()
print(f"Coherence Score: {coherence_score:.4f}")

## Perplexity 

**Perplexity** is a measure of how well a topic model can **explain the words** in a new document.

* It uses the **words in the document** to guess which topics are present.
* Then it checks how **likely those words** are, based on the **learned topic-word probabilities**.
* It does **not care about word order** — only which words appear and how often.

**Lower perplexity = better fit**
(The model is less “surprised” by the document’s words.)

Even if the document has **multiple topics**, the perplexity can still be low — as long as the topics match well with the document’s words.

---

- Perplexity is the exponential of the negative average log-likelihood per word
- Typical perplexity values for LDA models are usually in the range of 100–1000
- Lower values (e.g., < 100) indicate better generalization (less surprise),
- However, very low perplexity on the training set (e.g., < 50) can be a sign of overfitting,
meaning the model fits the training data too closely and may not generalize well to unseen data
- Very high values (e.g., > 1000) suggest poor topic modeling or an inappropriate number of topics
  
**Perplexity Formula:**

If `log_perplexity` is the negative average log-likelihood per word (from Gensim):

Perplexity = e^(-log_perplexity)

Where:
- `log_perplexity` is returned by `lda_model.log_perplexity(corpus)`
- `exp` is the exponential function (base *e*)

| log_perplexity | Actual perplexity | Interpretation                  |
|----------------|-------------------|---------------------------------|
| -5             | ~148              | Very good fit                   |
| -6             | ~403              | Good                            |
| -7             | ~1097             | Acceptable to borderline high   |
| -8             | ~2980             | Likely too high → poor generalization |

In [ ]:
perplexity = lda_model.log_perplexity(my_corpus)
print(f"Perplexity: {perplexity:.4f}")

# Visualize LDA topics using pyLDAvis

In [ ]:
# !pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare the visualization
vis_data = gensimvis.prepare(lda_model, my_corpus, my_dictionary)

# Set the figure size for better visualization
pyLDAvis.enable_notebook()

# Display the interactive visualization
pyLDAvis.display(vis_data)

In [ ]:
# Get 10 random documents and print their topics
import random
import numpy as np

# Select 10 random document indices
random_doc_indices = random.sample(range(len(my_corpus)), 10)

print("\nTopic Distribution for 10 Random Documents:")
print("-" * 50)

for idx in random_doc_indices:
    # Get the document's topic distribution
    doc_topics = lda_model.get_document_topics(my_corpus[idx])
    
    # Sort topics by probability (highest first)
    doc_topics = sorted(doc_topics, key=lambda x: x[1], reverse=True)
    
    # Get the original text (if available)
    original_text = df_summary.iloc[idx]
    
    print(f"\nDocument {idx}: \"{original_text}\"")
    print("Topic Distribution:")
    
    for topic_id, prob in doc_topics[:3]:
        # Get the top words for this topic
        topic_words = lda_model.show_topic(topic_id, topn=5)
        words = ", ".join([word for word, _ in topic_words])
        
        # Format the probability as a percentage
        prob_percent = prob * 100
        
        print(f"  Topic {topic_id+1}: {prob_percent:.2f}% ({words})")